In [9]:
import os
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o"
# os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

from crewai.tools import tool
import yfinance as yf
from crewai import Agent, Task, Crew, Process
from crewai_tools import ScrapeWebsiteTool
from langchain_openai import ChatOpenAI

# --- Tools 정의 ---
class Tools:
    @staticmethod
    @tool("One month stock price history")
    def stock_price(ticker: str) -> str:
        """
       Get one month's worth of historical stock price data for a given ticker as CSV.
        """
        stock = yf.Ticker(ticker)
        return stock.history(period="1mo").to_csv()

    @staticmethod
    @tool("Stock news URLs")
    def stock_news(ticker: str) -> list[str]:
        """
       Fetch recent news article URLs related to the given stock ticker.
        """
        stock = yf.Ticker(ticker)
        return [article["link"] for article in stock.news]

    @staticmethod
    @tool("Company's income statement")
    def income_stmt(ticker: str) -> str:
        """
       Get the income statement of a company as CSV for the given ticker.
        """
        stock = yf.Ticker(ticker)
        return stock.income_stmt.to_csv()

    @staticmethod
    @tool("Balance sheet")
    def balance_sheet(ticker: str) -> str:
        """
       Get the balance sheet of a company as CSV for the given ticker.
        """
        stock = yf.Ticker(ticker)
        return stock.balance_sheet.to_csv()

    @staticmethod
    @tool("Get insider transactions")
    def insider_transactions(ticker: str) -> str:
        """
       Retrieve insider trading transactions for the given stock ticker as CSV.
        """
        stock = yf.Ticker(ticker)
        return stock.insider_transactions.to_csv()


# --- Agents 정의 ---
class Agents:
    def technical_analyst(self):
        return Agent(
            role="Technical Analyst",
            goal="Analyses the movements of a stock and provides insights on trends, entry points, resistance and support levels.",
            backstory="An expert in technical analysis, you're known for your ability to predict stock movements and trends based on historical data.",
            verbose=True,
            tools=[Tools.stock_price],
            memory=True,
        )

    def researcher(self):
        return Agent(
            role="Researcher",
            goal="Gathers and summarizes news and sentiment around a stock.",
            backstory="You're skilled at collecting and analyzing sentiment and news related to a stock.",
            verbose=True,
            tools=[Tools.stock_news, ScrapeWebsiteTool()],
            memory=True,
        )

    def financial_analyst(self):
        return Agent(
            role="Financial Analyst",
            goal="Evaluates a stock's financial health using statements and insider data.",
            backstory="Experienced in financial statement analysis, you help investors understand the company’s valuation and performance.",
            verbose=True,
            tools=[
                Tools.income_stmt,
                Tools.balance_sheet,
                Tools.insider_transactions,
            ],
            memory=True,
        )

    def hedge_fund_manager(self):
        return Agent(
            role="Hedge Fund Manager",
            goal="Make strategic investment recommendations using data from analysts.",
            backstory="A seasoned fund manager making decisions based on research, technical, and financial insights.",
            verbose=True,
            memory=True,
        )


# --- Tasks 정의 ---
class Tasks:
    def research(self, agent):
        return Task(
            description="Gather and analyze the latest news and market sentiment surrounding the stock of {company}.",
            expected_output="Detailed summary of the news and market sentiment with potential implications.",
            agent=agent,
            output_file="stock_news.md",
        )

    def technical_analysis(self, agent):
        return Task(
            description="Perform technical analysis on {company}'s stock using price movements and trends.",
            expected_output="Technical report including support/resistance, patterns, entry points, and price targets.",
            agent=agent,
            output_file="technical_analysis.md",
        )

    def financial_analysis(self, agent):
        return Task(
            description="Analyze {company}'s financial statements and insider activity to assess financial health and valuation.",
            expected_output="Comprehensive financial report including revenue, earnings, valuation, and insider trades.",
            agent=agent,
            output_file="financial_analysis.md",
        )

    def investment_recommendation(self, agent, context):
        return Task(
            description="Provide a BUY or SELL recommendation for {company}'s stock based on all prior analyses.",
            expected_output="Final investment recommendation with risk/reward analysis and rationale.",
            agent=agent,
            context=context,
            output_file="investment_recommendation.md",
        )


# --- 실행 ---
agents = Agents()
tasks = Tasks()

researcher = agents.researcher()
technical_analyst = agents.technical_analyst()
financial_analyst = agents.financial_analyst()
hedge_fund_manager = agents.hedge_fund_manager()

research_task = tasks.research(researcher)
technical_task = tasks.technical_analysis(technical_analyst)
financial_task = tasks.financial_analysis(financial_analyst)
recommend_task = tasks.investment_recommendation(
    hedge_fund_manager,
    context=[
        research_task,
        technical_task,
        financial_task,
    ],
)

crew = Crew(
    agents=[
        researcher,
        technical_analyst,
        financial_analyst,
        hedge_fund_manager,
    ],
    tasks=[
        research_task,
        technical_task,
        financial_task,
        recommend_task,
    ],
    verbose=True,
    process=Process.hierarchical,
    manager_llm=ChatOpenAI(model="gpt-4o"),
    memory=True,
)

company = "Salesforce"

# description 내 {company} 변수 직접 format
research_task.description = research_task.description.format(company=company)
technical_task.description = technical_task.description.format(company=company)
financial_task.description = financial_task.description.format(company=company)
recommend_task.description = recommend_task.description.format(company=company)

# kickoff
result = crew.kickoff(inputs=dict(company=company))

print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 219997db-e406-4fd7-835a-2bac6282a2f5                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭────────────────────────────────────────────── 🧠 Retrieved Memory ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Historical Data:                                                                                               │
│  - Include actual news summaries and market sentiment analysis rather than just URLs.                           │
│  - Provide insights or implications based on the news to add depth.                                             │
│  - Ensure the output aligns more closely with the expected detailed summary.                                    │
│  - Include a comprehensive summary of the latest news articles about Salesforce.                                │
│  - Analyze recent market sentiment data and investor opinions.                                                  │
│  - Provide potential implications of the news and sentiment on Salesforce's stock.                              │
│  Recent Insights:                                                                                               │
│  - Thought: ...                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────── Retrieval Time: 1173.54ms ───────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Crew Manager                                                                                            │
│                                                                                                                 │
│  Task: Gather and analyze the latest news and market sentiment surrounding the stock of Salesforce.             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

/Users/nks/Documents/crew-ai/env/lib/python3.12/site-packages/chromadb/types.py:144: PydanticDeprecatedSince211: 
Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute 
from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x

/Users/nks/Documents/crew-ai/env/lib/python3.12/site-packages/chromadb/types.py:144: PydanticDeprecatedSince211: 
Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute 
from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 8d66fe0a-8666-4d08-a5df-514b1eac82c6                                                                     │
│  Agent: Crew Manager                                                                                            │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 219997db-e406-4fd7-835a-2bac6282a2f5                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: **Investment Recommendation: BUY Salesforce (CRM) Stock**                                        │
│                                                                                                                 │
│  **Rationale:**                                                                                                 │
│                                                                                                                 │
│  1. **Fundamental Strength:**                                                                                   │
│     - Salesforce has demonstrated robust financial performance with substantial revenue growth from $26.5       │
│  billion in 2022 to $37.9 billion in 2025. Net income has significantly increased, indicating improved          │
│  profitability and operational efficiencies.                                                                    │
│     - The balance sheet shows a stable position with increasing total assets and stockholders' equity,          │
│  suggesting financial stability.                                                                                │
│                                                                                                                 │
│  2. **Strategic Initiatives:**                                                                                  │
│     - Salesforce's focus on integrating artificial intelligence, expanding cloud services, and strategic        │
│  acquisitions positions it well for future growth. These initiatives are essential drivers for maintaining its  │
│  competitive edge and expanding its market reach.                                                               │
│                                                                                                                 │
│  3. **Market Sentiment:**                                                                                       │
│     - Overall positive market sentiment is supported by consistent revenue growth and an expanding customer     │
│  base. While macroeconomic uncertainties exist, Salesforce’s proactive adaptation strategies bolster investor   │
│  confidence.                                                                                                    │
│                                                                                                                 │
│  4. **Technical Analysis:**                                                                                     │
│     - The stock is in a consolidation phase with defined support around $257-$259 and resistance near           │
│  $274-$276. This presents a potential buying opportunity near support with a short-term target near             │
│  resistance.                                                                                                    │
│     - Technical indicators such as RSI and MACD suggest neutral momentum, supporting a cautious but optimistic  │
│  approach.                                                                                                      │
│                                                                                                                 │
│  5. **Risk Considerations:**                          

**Investment Recommendation: BUY Salesforce (CRM) Stock**

**Rationale:**

1. **Fundamental Strength:**
   - Salesforce has demonstrated robust financial performance with substantial revenue growth from $26.5 billion in 2022 to $37.9 billion in 2025. Net income has significantly increased, indicating improved profitability and operational efficiencies.
   - The balance sheet shows a stable position with increasing total assets and stockholders' equity, suggesting financial stability.

2. **Strategic Initiatives:**
   - Salesforce's focus on integrating artificial intelligence, expanding cloud services, and strategic acquisitions positions it well for future growth. These initiatives are essential drivers for maintaining its competitive edge and expanding its market reach.

3. **Market Sentiment:**
   - Overall positive market sentiment is supported by consistent revenue growth and an expanding customer base. While macroeconomic uncertainties exist, Salesforce’s proactive adaptation str